<a href="https://colab.research.google.com/github/ramonferrari/mpgc_ti/blob/main/BDADS_2S2023_Tarefa_Aulas_5_e_6_cluster.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tarefa Aulas 5 e 6
# Exploração de Cluster Analysis
#### Grupo:
1.   Antonio Angelo dos Santos
2. Gabriel Menegon Cardozo
3. Gustavo Salomão
4. Lucas Felipe Ballesteros Martins
5. Ramon Moreno Ferrari
6. Valter Bellei



### Preparação: instala e carrega pacotes necessários

In [9]:
install.packages("ggplot2")
install.packages("ggdendro")
install.packages("stringr")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [10]:
suppressWarnings(suppressMessages(library(ggplot2)))
suppressWarnings(suppressMessages(library(ggdendro)))
suppressWarnings(suppressMessages(library(stringr)))

## Exploração de Cluster Analysis

In [11]:
# Endereço do arquivo na nuvem, para funcionar para todos os colegas, em qualquer computador
url_dataset <- c("https://raw.githubusercontent.com/ramonferrari/mpgc_ti/main/Sanduiches.csv")

In [16]:
# Leitura dos dados
sanduiches <- read.table(url_dataset,header=TRUE,sep = ",")

# Deixa variáveis com nomes bonitinhos porque temos TOC
names(sanduiches) <- sub("\\.", " ", names(sanduiches))
sanduiches$`Sanduíches` <- str_to_title(sanduiches$`Sanduíches`)
names(sanduiches)

# Dataset lido corretamente?
head(sanduiches)

[1] "Sanduíches"       "Valor Energético" "Carboidratos"     "Proteínas"       
 [5] "Gordura Total"    "Gordura Saturada" "Gordura Trans"    "Colesterol"      
 [9] "Fibra Alimentar"  "Sódio"            "Cálcio"           "Ferro"

,Sanduíches,Valor Energético,Carboidratos,Proteínas,Gordura Total,Gordura Saturada,Gordura Trans,Colesterol,Fibra Alimentar,Sódio,Cálcio,Ferro
,<chr>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<int>,<dbl>
1,Big Mac,476,41,25,24,9.3,0.5,64,2.7,898,176,5.4
2,Big Tasty,841,43,42,55,23.0,1.9,109,3.1,1454,335,7.4
3,Quarterão,539,35,31,31,15.0,1.5,88,2.7,979,238,5.5
4,Mcnífico Bacon,563,37,32,32,13.0,1.3,86,2.3,929,166,5.8
5,Cheddar Mcmelt,484,30,29,27,13.0,1.2,80,2.3,844,150,5.2
6,Mcfish,363,37,17,16,4.6,0.4,40,2.5,534,149,3.9


In [17]:
hc <- hclust(dist(sanduiches), "average")

Warning message in dist(sanduiches):
“NAs introduced by coercion”
